In [1]:
#os and i/o
import os
import numpy as np
import glob
from os.path import abspath
import csv

#scientific computing
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
from scipy import stats, optimize
from pandas import DataFrame, Series
import seaborn as sns
import random as rd
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm
import scipy.stats

#ipython add-ons
from IPython.parallel import Client
from IPython.display import Image

##nipype
import nibabel as nib
from nipype.pipeline.engine import Node, MapNode, Workflow
from nipype.interfaces.io import DataGrabber, DataFinder, DataSink
from nipype.interfaces import fsl
from nipype.interfaces.fsl import BET
from nipype.interfaces.freesurfer.preprocess import ReconAll
from nipype.interfaces.freesurfer.utils import MakeAverageSubject
%matplotlib inline

In [2]:
#preliminary housekeeping
home_dir = '/data/home/iballard/fd/'
subj_file = home_dir + 'scripts/sub_cb_mappings.txt'
os.chdir(home_dir)

num_slices = 57 #if you don't know, use nib.load and shape attribute
#get subject list
sub_list = []
with open(subj_file) as f:
    for line in f:
        (key,val) = line.strip().split(' ') 
        sub_list.append('fd_' + key)

In [13]:
from nipype.interfaces.fsl import ExtractROI
##add a z slice to all scans
for sub in sub_list:
    print sub
    scans = glob.glob('data/' + sub + '/cal/*') #calibration scans
    scans.extend(glob.glob('data/' + sub + '/func/*/*.nii.gz')) #functional scans
    for scan in scans:
        old_file = abspath(scan)
        new_file = abspath(scan[:-7] + '_' + str(num_slices + 1)) + '.nii.gz'
        fslroi = ExtractROI(in_file=old_file, roi_file=new_file,
                            x_min=0,x_size=-1,y_min=0,y_size=-1,
                            z_min=0,z_size= (num_slices + 1))
        fslroi.run()
        

fd_101


KeyboardInterrupt: 

In [17]:
from nipype.interfaces.fsl import ExtractROI
from IPython.parallel import Client
rc = Client()
print rc.ids()
dview = rc[:]
##add a z slice to all scans

# for sub in sub_list:
#     print sub
#     scans = glob.glob('data/' + sub + '/cal/*') #calibration scans
#     scans.extend(glob.glob('data/' + sub + '/func/*/*.nii.gz')) #functional scans
#     for scan in scans:
#         old_file = abspath(scan)
#         new_file = abspath(scan[:-7] + '_' + str(num_slices + 1)) + '.nii.gz'
#         fslroi = ExtractROI(in_file=old_file, roi_file=new_file,
#                             x_min=0,x_size=-1,y_min=0,y_size=-1,
#                             z_min=0,z_size= (num_slices + 1))
#         fslroi.run()
        

TimeoutError: Hub connection request timed out

In [ ]:
from nipype.pipeline.engine import Node, MapNode, Workflow
from nipype.interfaces.io import DataGrabber, DataFinder, DataSink
from nipype.interfaces import fsl
from nipype.interfaces.fsl import BET
from nipype.interfaces.fsl import ExtractROI
from os.path import abspath
from IPython.display import Image

os.chdir('/home/iballard/')
add_slice = Node(ExtractROI, name="add_slice")

##set up data grabber
ds = Node(DataGrabber(infields=['subject_id','task_id','run_id'],outfields =['func','cal1','cal2']),
         name = 'datasource')
ds.inputs.base_directory = abspath('fd/data/')
ds.inputs.template = '*'
ds.inputs.template_args = {'func': [['subject_id','task_id','run_id']], 'cal1':[['subject_id']],
                          'cal2':[['subject_id']]}
ds.inputs.field_template = {'func': '%s/func/%s/run_%s.nii.gz' ,
                      'anat': '%s/anat/T1w_Whole_brain_1mm.nii.gz'}
ds.inputs.sort_filelist = True

ds.inputs.subject_id = 'fd_101'
ds.inputs.task_id = '*'
ds.inputs.run_id = 2
ds.iterables = ('subject_id',sub_list)
print ds.run().outputs

wf = Workflow(name = 'groupstrip')
wf.basedir = abspath('fd/data/')
wf.connect(ds,'anat',skullstrip,'in_file')

##set up Data Sink
sinker = Node(DataSink(), name = 'sinker')
sinker.inputs.base_directory = abspath('output')
wf.connect(skullstrip,'out_file',sinker,'skull_stripped')

#visualize workflow
wf.write_graph('workflow_graph.dot')
Image(filename = 'workflow_graph.dot.png' )

In [32]:
##merge the two gradient reversal scans
identifier = str(num_slices + 1) #file identifier
for sub in sub_list:
    if sub == 'fd_101'
    scans = glob.glob('data/' + sub + '/cal/*' + identifier + '*') #padded calibration scans
    for scan in scans:
        old_file = abspath(scan)
        new_file = abspath(scan[:-7] + '_' + str(num_slices + 1))
        fslroi = ExtractROI(in_file=old_file, roi_file=new_file,
                            x_min=1,x_size=1)
        